In [1]:
from scipy.sparse import csr_matrix, dok_matrix, coo_matrix
import pandas as pd
import numpy as np
import implicit
from tqdm.notebook import tqdm

In [33]:
df = pd.read_csv("/w/225/1/chess/tactics/prediction_data_full_shuffled.csv")
df

,Unnamed: 0,user_hash,puzzle_1,puzzle_2,cluster_1,rating_labels,rating_1,cluster_2,rating_2,cluster_perf_1,cluster_perf_2,passed_puzzle
0,0,0df86500b477547f62eea54bb9e136ae81d75ad924090a...,34864,195592,6,2,407,14,420,807.000000,1072.145455,0
1,1,2e3f96438c113cb00b724c2b0b5399ed66bc7451207a63...,195592,34864,14,2,420,6,407,1043.076760,808.900000,1
2,2,e7be030b30d68734fe1bfd8801bb4104e81db94d57f871...,195592,34864,14,2,420,6,407,1205.652174,759.000000,1
3,3,a9797f35d72e6473da05de5176750b5f836fbc414a37f5...,195592,34864,14,2,420,6,407,1246.872996,909.777778,1
4,4,7af510b2c63044cef2379aae31e6cb880816e987217dfc...,195592,39933,14,2,420,5,411,1188.519048,1638.991643,1
...,...,...,...,...,...,...,...,...,...,...,...,...
806326,806326,01ad240792e4b24aa32ec68f4589b54dcc3415c570fdf8...,97236,158723,9,1,400,8,400,477.538462,538.833333,0
806327,806327,01ad240792e4b24aa32ec68f4589b54dcc3415c570fdf8...,185109,158723,9,1,400,8,400,477.538462,538.833333,0
806328,806328,01ad240792e4b24aa32ec68f4589b54dcc3415c570fdf8...,158723,158103,8,1,400,9,400,538.833333,477.538462,1
806329,806329,6fb74b461ab717b4ab25eae19702bc944209a7ce09576f...,153150,79682,8,4,451,17,462,567.583333,627.617647,0


In [5]:
#baseline prediction based on ELO rating by assuming the higher rating is the successful one.

r,c = df.shape

correct_prediction = 0
for i in range(r):
    rating_1 = df.iloc[i,6]
    rating_2 = df.iloc[i,8]
    if rating_1 >= rating_2:
        predict_puzzle = "puzzle_1"
    else:
        predict_puzzle = "puzzle_2"
    
    if df.iloc[i,11] == 0:
        correct_puzzle = "puzzle_1"
    else:
        correct_puzzle = "puzzle_2"
    
    if correct_puzzle == predict_puzzle:
        correct_prediction += 1

correct_prediction / r
    

0.49679226025044304

In [67]:
#baseline prediction based on ELO rating by assuming the lower rating is the successful one.

r,c = df.shape

correct_prediction = 0
for i in range(r):
    rating_1 = df.iloc[i,6]
    rating_2 = df.iloc[i,8]
    if rating_1 <= rating_2:
        predict_puzzle = "puzzle_1"
    else:
        predict_puzzle = "puzzle_2"
    
    if df.iloc[i,11] == 0:
        correct_puzzle = "puzzle_1"
    else:
        correct_puzzle = "puzzle_2"
    
    if correct_puzzle == predict_puzzle:
        correct_prediction += 1

correct_prediction / r
    

0.5032077397495569

In [52]:
#baseline prediction based on ELO rating by randomly choosing the successful one.

r,c = df.shape

sum_accuracy = 0
repeat = 10

for n in range(repeat):
    correct_prediction = 0
    for i in range(r):
        predict_puzzle = np.random.randint(1,3)
        if predict_puzzle == 1:
            predict_puzzle = "puzzle_1"
        else:
            predict_puzzle = "puzzle_2"

        if df.iloc[i,11] == 0:
            correct_puzzle = "puzzle_1"
        else:
            correct_puzzle = "puzzle_2"

        if correct_puzzle == predict_puzzle:
            correct_prediction += 1
    accuracy = correct_prediction / r
    sum_accuracy += accuracy

avg_accuracy = sum_accuracy / repeat
avg_accuracy    

0.49997916488390004

In [53]:
#baseline prediction based on cluster rating by assuming the higher cluster performance is the successful one.

r,c = df.shape

correct_prediction = 0
for i in range(r):
    cluster_perf_1 = df.iloc[i,9]
    cluster_perf_2 = df.iloc[i,10]
    predict_puzzle = np.random.randint(1,3)
    if predict_puzzle == 1:
        predict_puzzle = "puzzle_1"
    else:
        predict_puzzle = "puzzle_2"
    
    if df.iloc[i,11] == 0:
        correct_puzzle = "puzzle_1"
    else:
        correct_puzzle = "puzzle_2"
    
    if correct_puzzle == predict_puzzle:
        correct_prediction += 1

correct_prediction / r
    

0.5124483617769874

In [55]:
#baseline prediction based on cluster rating by randomly choosing the successful one.

r,c = df.shape

sum_accuracy = 0
repeat = 10

for n in range(repeat):
    correct_prediction = 0
    for i in range(r):
        predict_puzzle = np.random.randint(1,3)
        if predict_puzzle == 1:
            predict_puzzle = "puzzle_1"
        else:
            predict_puzzle = "puzzle_2"

        if df.iloc[i,11] == 0:
            correct_puzzle = "puzzle_1"
        else:
            correct_puzzle = "puzzle_2"

        if correct_puzzle == predict_puzzle:
            correct_prediction += 1
    accuracy = correct_prediction / r
    sum_accuracy += accuracy

avg_accuracy = sum_accuracy / repeat
avg_accuracy 

0.49984237242521984

In [10]:
#reformatting the df with the structure as below. This is done so that it's easier to create the training data for BPR.

data = pd.DataFrame(index = np.arange(r*2), columns = ['user_hash','puzzle_id', 'is_passed'])

for i in tqdm(range(r)):
    data.iloc[2*i,0] = df.iloc[i,1]
    data.iloc[2*i,1] = df.iloc[i,2]
    data.iloc[2*i+1,0] = df.iloc[i,1]
    data.iloc[2*i+1,1] = df.iloc[i,3]
    if df.iloc[i,11] == 0:
        data.iloc[2*i,2] = 1
        data.iloc[2*i+1,2] = 0
    else:
        data.iloc[2*i,2] = 0
        data.iloc[2*i+1,2] = 1

data

,user_hash,puzzle_id,is_passed
0,0df86500b477547f62eea54bb9e136ae81d75ad924090a...,34864,1
1,0df86500b477547f62eea54bb9e136ae81d75ad924090a...,195592,0
2,2e3f96438c113cb00b724c2b0b5399ed66bc7451207a63...,195592,0
3,2e3f96438c113cb00b724c2b0b5399ed66bc7451207a63...,34864,1
4,e7be030b30d68734fe1bfd8801bb4104e81db94d57f871...,195592,0
...,...,...,...
1612657,01ad240792e4b24aa32ec68f4589b54dcc3415c570fdf8...,158103,1
1612658,6fb74b461ab717b4ab25eae19702bc944209a7ce09576f...,153150,1
1612659,6fb74b461ab717b4ab25eae19702bc944209a7ce09576f...,79682,0
1612660,c98995bbf2a8a14ed62163506ee1b650e8d5d5c970bc3f...,314989,0


In [2]:
data = data.to_csv("prediction_data.csv", index = False)

NameError: name 'data' is not defined

In [2]:
data = pd.read_csv("prediction_data.csv")

In [56]:
def create_matrix(data, users_col, items_col, ratings_col, option, threshold = None):
    """
    creates the sparse user-item interaction matrix,
    if the data is not in the format where the interaction only
    contains the positive items (indicated by 1), then use the 
    threshold parameter to determine which items are considered positive
    
    Parameters
    ----------
    data : DataFrame
        implicit rating data

    users_col : str
        user column name

    items_col : str
        item column name
    
    ratings_col : str
        implicit rating column name

    threshold : int, default None
        threshold to determine whether the user-item pair is 
        a positive feedback

    Returns
    -------
    ratings : scipy sparse csr_matrix, shape [n_users, n_items]
        user/item ratings matrix

    data : DataFrame
        implict rating data that retains only the positive feedback
        (if specified to do so)
    """
    #if threshold is not None:
        #data = data[data[ratings_col] >= threshold]
        #data[ratings_col] = 1
    
    for col in (items_col, users_col, ratings_col):
        data[col] = data[col].astype('category')
    
    if option == "item_users":
        ratings = coo_matrix((data[ratings_col],
                              (data[items_col].cat.codes, data[users_col].cat.codes)))
    else:
        ratings = csr_matrix((data[ratings_col],
                              (data[users_col].cat.codes, data[items_col].cat.codes)))
    ratings.eliminate_zeros()
    
    """item_to_code = {}
    puzzle_id_list = list(data['puzzle_id'])
    code_item_list = list(data[items_col].cat.codes)
    for i in range(len(puzzle_id_list)):
        item_to_code[puzzle_id_list[i]] = code_item_list[i]
        
    user_to_code = {}
    user_hash_list = list(data['user_hash'])
    code_user_list = list(data[users_col].cat.codes)
    for i in range(len(user_id_list)):
        user_to_code[user_hash_list[i]] = code_user_list[i]"""
    
    #return ratings, data, user_to_code, item_to_code
    return ratings, data

In [57]:
items_col = 'puzzle_id'
users_col = 'user_hash'
ratings_col = 'is_passed'
threshold = 1
item_user_data, data_filtered  = create_matrix(data, users_col, items_col, ratings_col, "item_users", threshold)
user_item_data, data_filtered = create_matrix(data, users_col, items_col, ratings_col, "user_items", threshold)

In [58]:
model = implicit.bpr.BayesianPersonalizedRanking()
model.fit(item_user_data)

In [59]:
#create a dictionary that maps the puzzle id to its respective numerical code.

item_to_code = {}
puzzle_id_list = list(data['puzzle_id'])
code_item_list = list(data[items_col].astype("category").cat.codes)
for i in range(len(puzzle_id_list)):
    item_to_code[puzzle_id_list[i]] = code_item_list[i]

#create a dictionary that maps the user hash to its respective numerical code.

user_to_code = {}
user_hash_list = list(data['user_hash'])
code_user_list = list(data[users_col].astype("category").cat.codes)
for i in range(len(user_hash_list)):
    user_to_code[user_hash_list[i]] = code_user_list[i]

In [60]:
#predicting "ratings" for specific user. The values are in the range of [0,1]

user_pred = model.user_factors[user_to_code[user_hash_list[0]]].dot(model.item_factors.T)
user_pred.shape

(20552,)

In [61]:
model.item_factors.shape

(20552, 101)

In [62]:
item_user_data.shape

(20552, 288)

In [37]:
#training accuracy of BPR ranking (yeah, this is wrong. I can't figure out how to do the evaluation properly.)

r,c = df.shape

correct_prediction = 0
for i in range(r):
    item_id_list = [item_to_code[df.iloc[i,2]], item_to_code[df.iloc[i,3]]]
    rank_items = model.rank_items(user_to_code[user_hash_list[i]], user_item_data, item_id_list)
    predict_puzzle = rank_items[0][0]
    
    if df.iloc[i,11] == 0:
        correct_puzzle = item_to_code[df.iloc[i,2]]
    else:
        correct_puzzle = item_to_code[df.iloc[i,3]]
    
    if correct_puzzle == predict_puzzle:
        correct_prediction += 1

correct_prediction / r
    

0.48717462183644183